In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
import pandas as pd
from pathlib import Path
from PIL import Image
import cv2
import time
import os
from tensorflow import keras
from tensorflow.keras.preprocessing import image_dataset_from_directory
import random
import shutil

In [ ]:
count = 0
data = []
test_data_path = []
basepath = Path('faces_data/')
files_in_basepath = basepath.iterdir()
for item in files_in_basepath:
    files_in_subpath = item.iterdir()
    for item1 in files_in_subpath:
        data.append(item1)
    random.shuffle(data)
    test_data_path.append(data[0:int(np.round((0.1 * len(data))))])
    data = []
test_data_path = list(np.concatenate(test_data_path).flat)

In [ ]:
test_data = []
test_labels = []
count = 0
for face in test_data_path:
    count = count+1
    path = str(Path(face))
    test_data.append(cv2.imread(path)[:,:,::-1])
    test_labels.append(str(face).split('/')[1])
    #shutil.move(path, 'test_data_face/' + str(count) + '.jpg')
test_data = np.array(test_data)
test_data = (test_data-127.5)/127.5

In [ ]:
np.shape(test_data)

In [ ]:
train_data = image_dataset_from_directory(
    directory='faces_data/',
    labels='inferred',
    label_mode='categorical',
    subset="training",
    validation_split = 0.1,
    seed=123,
    batch_size=64,
    image_size=(128, 128))

In [ ]:
validation_data = image_dataset_from_directory(
    directory='faces_data/',
    labels='inferred',
    label_mode='categorical',
    subset="validation",
    validation_split = 0.1,
    seed=123,
    batch_size=64,
    image_size=(128, 128))

In [ ]:
classes = train_data.class_names

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE
train_data = train_data.cache().prefetch(buffer_size=AUTOTUNE)
validation_data = validation_data.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
strategy = tf.distribute.MirroredStrategy()
print('Number of devices: {}'.format(strategy.num_replicas_in_sync))

In [ ]:
with strategy.scope():

    model = tf.keras.Sequential()
    model.add(tf.keras.layers.InputLayer(input_shape=(128,128,3)))
    model.add(tf.keras.layers.Rescaling(1./127.5, offset=-1))
    model.add(tf.keras.applications.ResNet50(include_top = False, weights = 'imagenet', pooling = 'max', input_shape= (128,128,3)))
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(1225, activation = 'softmax'))
    model.add(tf.keras.layers.Dropout(0.5))
    model.summary()
    
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate = 0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
import wandb
wandb.init()

In [ ]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', mode = 'max', patience = 3)

history = model.fit(train_data, epochs=100, batch_size = 64, validation_data = validation_data, callbacks = [early_stopping], verbose=1)

In [ ]:
history_df = pd.DataFrame(history.history)
history_df.head()

In [ ]:
# plotting training and validation loss
plt.title('Loss')
plt.plot(history_df['loss'], label = 'Training loss')
plt.plot(history_df['val_loss'], label = 'Validation loss')
plt.legend()
plt.show()

In [ ]:
# plotting training and validation accuracy
plt.title('Accuracy')
plt.plot(history_df['accuracy'], label = 'Training accuracy')
plt.plot(history_df['val_accuracy'], label = 'Validation accuracy')
#plt.ylim(0,1)
plt.legend()
plt.show()

In [ ]:
model.save('/home/raghav/Downloads/classifier.h5')

In [ ]:
'''my_model = tf.keras.models.load_model('/home/raghav/Downloads/classifier.h5')'''

In [ ]:
predY = model.predict(testX)
predY = np.round(predY, decimals = 0)
accuracy = accuracy_score(testY, predY)
print("Test accuracy - ", (accuracy*100))

In [ ]:
data_df = pd.read_csv("final_data_path.csv")
data_df